In [1]:
%%capture
## [TEMPLATE BEGIN]
## おまじない：実行中のノートブック名からOUTPUT_DIRを決めるコード
from pathlib import Path
from IPython import get_ipython; ip = get_ipython()
if '__vsc_ipynb_file__' in ip.user_ns:
    ## VS Codeで実行されている場合
    THIS_NOTEBOOK_PATH = Path(ip.user_ns['__vsc_ipynb_file__'])
else:
    !pip install ipynbname
    import ipynbname; THIS_NOTEBOOK_PATH = Path(ipynbname.path())
    if "%2F" in THIS_NOTEBOOK_PATH.stem:
        ## Google Colabで実行されている場合
        colab_ip = %system hostname -I   # uses colab magic to get list from bash
        colab_ip = colab_ip[0].strip()   # returns "172.28.0.12"
        colab_port = 9000                # could use 6000, 8080, or 9000
        import requests
        filename = requests.get(f"http://{colab_ip}:{colab_port}/api/sessions").json()[0]["name"]
        THIS_NOTEBOOK_PATH = Path(filename)
    else:
        ## それ以外の場合
        THIS_NOTEBOOK_PATH = THIS_NOTEBOOK_PATH

OUTPUT_DIR = Path("figures") / THIS_NOTEBOOK_PATH.stem
## [TEMPLATE END]

In [ ]:
print("File output directory:", OUTPUT_DIR)
# Create the output directory if it does not exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.patches as mpatches
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
import numpy as np

# Set a random seed for reproducibility
RANDOM_STATE = 42

# Fetch the wine dataset from sklearn
data = load_wine(as_frame=True)
df = data.frame

# Select only the specified features
features = ['alcohol', 'malic_acid']
X = df[features]
y = df['target']
target_names = ['Barolo', 'Barbera', 'Grignolino']  # Custom class names

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train decision tree classifier
clf = DecisionTreeClassifier(max_depth=3, random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

# Create a mesh to plot the decision boundary
x_min, x_max = X_train.iloc[:, 0].min() - 1, X_train.iloc[:, 0].max() + 1
y_min, y_max = X_train.iloc[:, 1].min() - 1, X_train.iloc[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# Correcting the indexing error
# Update the target values to reflect custom class names
target_names = ['Barolo', 'Barbera', 'Grignolino']
y_train_named = [target_names[i] for i in y_train]
y_test_named = [target_names[i] for i in y_test]

# Create a new color map for the custom class names
custom_colors = {'Barolo': 'red', 'Barbera': 'blue', 'Grignolino': 'green'}
cmap_custom = ListedColormap([custom_colors[name] for name in target_names])

# Predict the classifier output for the mesh using the original target values for consistency in Z
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the decision boundary with custom class names
plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=cmap_custom)
for target, color in custom_colors.items():
    idx_train = [i for i, t in enumerate(y_train_named) if t == target]
    idx_test = [i for i, t in enumerate(y_test_named) if t == target]
    plt.scatter(X_train.iloc[idx_train, 0], X_train.iloc[idx_train, 1], c=color, marker='o', edgecolor='k', s=100, label=f"Training set ({target})")
    plt.scatter(X_test.iloc[idx_test, 0], X_test.iloc[idx_test, 1], c=color, marker='x', edgecolor='k', s=100, label=f"Test set ({target})")
plt.xlabel(features[0])
plt.ylabel(features[1])
plt.legend(loc='upper left', scatterpoints=1)
plt.title('Decision Tree Classifier with Custom Class Names')

# Custom legend patches
patch0 = mpatches.Patch(color='red', label='Barolo')
patch1 = mpatches.Patch(color='blue', label='Barbera')
patch2 = mpatches.Patch(color='green', label='Grignolino')
plt.legend(handles=[patch0, patch1, patch2], loc='upper left')

plt.savefig(OUTPUT_DIR / "wine_decision_tree_segmentation.png", bbox_inches='tight', pad_inches=0)

plt.show()